In [1]:
#ollama 서버 실행중인지 확인
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3:latest")
llm.invoke("서울이란?")

AIMessage(content='😊\n\n"서울" (Seoul) is the capital city of South Korea. It is a megacity with a population of over 10 million people, making it one of the most populous cities in the world.\n\nSeoul is known for its rich history and culture, dating back to the 18th century when it was the capital of the Joseon Dynasty. Today, it is a modern city with a vibrant economy, cutting-edge technology, and a blend of traditional and contemporary architecture.\n\nSome popular attractions in Seoul include:\n\n1. Gyeongbokgung Palace: The largest and most iconic palace in Korea.\n2. Bukchon Hanok Village: A traditional Korean village with preserved hanok houses.\n3. Myeong-dong Shopping District: One of the busiest shopping areas in Asia.\n4. Namsan Tower (Seoul Tower): A popular spot for panoramic views of the city.\n5. Insadong Street: A famous street filled with traditional tea houses, shops, and restaurants.\n\nSeoul is also known for its food scene, with popular dishes like bibimbap, bulgogi

In [2]:
#prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "What is stock??"})

AIMessage(content="😊\n\n스톡(stock)입니다. STOCK는 회사의 주식이나 자본을 의미합니다. 즉, 특정 회사에 대한 지분이나 권리를 나타내는 증서입니다. 일반적으로 투자자들은股票을 구입하여 해당 회사의 성장으로부터 이익을 얻습니다. STOCK는 주로 주식시장에서 교환되며, 이를 통해 투자자들은 다양한 회사의 주식을 구입하거나 판매할 수 있습니다.\n\n(Translation: Stock is a company's stock or capital. It means the share or right to a specific company. Generally, investors buy stocks and gain benefits from the growth of that company. Stocks are traded on the stock market, allowing investors to purchase or sell shares of various companies.)", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:07:59.1804134Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1530939500, 'load_duration': 3622800, 'prompt_eval_count': 48, 'prompt_eval_duration': 47954000, 'eval_count': 157, 'eval_duration': 1477965000}, id='run-f3a4a1ea-b004-4644-97cc-a8560e688add-0')

In [3]:
#output parser사용
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "What is stock?"})

'😊\n\n스톡(stock)란 기업의 주식 또는 자본을 의미합니다. 즉, 특정 기업이 발행한 주식을 말하는 것입니다. 이 주식은 투자자들이 기업의 성장 가능성을 예상하여 구입하거나 판매할 수 있는 증권입니다.'

In [4]:
#체인 스트림 출력
chain = prompt | llm | StrOutputParser()
for token in chain.stream(
    {"input": "What is stock?"}
):
    print(token, end="")

😊

 stocks, or, , are shares of ownership in a company that represent a claim on part of its assets and profits. In other words, stocks give you a small piece of a company's equity, allowing you to participate in its growth and success.

Stocks can be traded on stock exchanges like the Korea Exchange (KRX) or over-the-counter markets, providing investors with an opportunity to buy and sell shares in various companies.

Do you have any other questions about stocks?

In [5]:
# 첫번째 체인
prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
chain1 = (
    prompt1 
    | llm 
    | StrOutputParser()
)
message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
print(f'message1: {message1}') # What is a stock?

# 두번째 체인
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer the question in Korean"),
    ("user", "{input}")
])
chain2 = (
    {"input": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)
message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
print(f'message2: {message2}') # 주식은 한 회사의 소유권을 나타내는 증권입니다. 즉, 특정 기업에 투자하여 (중략)

message1: "What is stock?"
message2: 😊

식권(stock)은 일정한 조건에 따라 발행된 주식을 의미합니다. 주식은 기업의 자산을 구성하는 부분 중 하나로, 주주가 지분을 가질 수 있는 증서를 말합니다. 주식은 주당 가격으로 매매되며, 주주는 해당 주식의 소유권을 갖게 됩니다.

(Translation: Stock refers to a certain type of security that is issued under specific conditions. Stocks are part of a company's assets and represent ownership stakes in the business. Stock prices are traded on an exchange, and shareholders own the corresponding stocks.)


In [6]:
#체인 병렬 실행
from langchain_core.runnables import RunnableParallel
joke_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 짧은 농담 말해줘") 
    | llm)
poem_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 시 2줄 써줘") 
    | llm)

# map_chain = {"joke": joke_chain, "poem": poem_chain} # 체인에서 이처럼 사용할 때, 자동으로 RunnableParallel 사용됨
# map_chain = RunnableParallel({"joke": joke_chain, "poem": poem_chain})
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "애플"})

{'joke': AIMessage(content="Here's a short Apple-related joke:\n\nWhy did the Apple employee break up with his girlfriend?\n\nBecause he needed more space... and a better battery life! 🤣🍎", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:08:15.8757367Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1078639400, 'load_duration': 1520900, 'prompt_eval_count': 14, 'prompt_eval_duration': 14051000, 'eval_count': 38, 'eval_duration': 336061000}, id='run-0856d868-7623-43a6-817f-e44dcc257b0f-0'),
 'poem': AIMessage(content="Here are two lines about Apple:\n\nInnovative designs and user-friendly interfaces have made Apple a household name, with the iPhone revolutionizing the way we communicate. From Macs to iPads, Apple's ecosystem has become an integral part of modern life, shaping the tech industry for decades to come.", response_metadata={'model': 'llama3:latest', 'created_at': '2024-07-18T10:08:15.5197738Z'